In [1]:
from feos_pcsaft.dft import PcSaftFunctional, Pore3D, State, DFTSolver, FMTVersion, Pore1D, ExternalPotential, Geometry
from feos_pcsaft import PcSaftParameters
from feos_pcsaft.si import *
from scipy.optimize import minimize, Bounds
import os
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import yt
#yt.toggle_interactivity()
from scipy.spatial import Voronoi, distance

In [ ]:
params = PcSaftParameters.from_json(['carbon-dioxide'],'20191105_pure_parameters.json')
func = PcSaftFunctional.new_full(params,FMTVersion.AntiSymWhiteBear)

RuntimeError: No such file or directory (os error 2)

In [4]:
class SolidStructure:

    N_atom = 0
    dimensions = {
        "Lx": 0.0,
        "Ly": 0.0,
        "Lz": 0.0
    }
    forcefield = ''
    total_mass = 0.0

    def __init__(self,structure):
        
        self.name = structure

        database = os.path.join(os.getcwd(),'structure_parameters','solid_database.json')
    
        with open(database) as f:
            data = json.load(f)

            for i in range(len(data)):
                if data[i]['Name']==self.name:
                    self.N_atom = data[i]['N_atom']
                    self.dimensions = data[i]['Dimensions']
                    self.forcefield = data[i]['Forcefield']
                         
    def read_structure(self):
        struct_param = os.path.join(os.getcwd(),'structure_parameters')
        structure_df = pd.read_csv(os.path.join(struct_param,'{}.dat'.format(self.name)),names=['x','y','z','Type','mask'], delim_whitespace=True)
        
        filename = os.path.join(struct_param,'{}.dat'.format(self.forcefield))
        forcefield_df = pd.read_csv(filename,names=['Type','sigma','epsilon','mass'], delim_whitespace=True)

        coordinates = np.array([structure_df["x"], structure_df["y"], structure_df["z"]])
        
        sigma_ss = np.zeros(len(structure_df["x"]))
        epsilon_k_ss = np.zeros_like(sigma_ss)
        
        for i in range(len(sigma_ss)):
            sigma_ss[i] = forcefield_df.sigma[forcefield_df.Type==structure_df["Type"][i]]
            epsilon_k_ss[i] = forcefield_df.epsilon[forcefield_df.Type==structure_df["Type"][i]]
        
        self.total_mass = np.sum(np.array([forcefield_df.mass[forcefield_df.Type==t]for t in structure_df.Type])) * 1.66054e-27

        return coordinates, sigma_ss, epsilon_k_ss, np.array(structure_df['mask'])

In [5]:
structure = SolidStructure('TpPA')
coordinates, sigma, epsilon, mask = structure.read_structure()
system_size = [structure.dimensions["Lx"] * ANGSTROM, structure.dimensions["Ly"] * ANGSTROM, structure.dimensions["Lz"] * ANGSTROM]

FileNotFoundError: [Errno 2] No such file or directory: '/usr/ITT/bursik/promotion/entropy_scaling_AS/entropyScaling/feos-pcsaft/examples/adsorption_processes/structure_parameters/solid_database.json'

In [5]:
%%time
vor = Voronoi(coordinates.T)

minimum_distance = 0.0
dist = np.zeros(int(vor.vertices.size / 2))
for i,point in enumerate(vor.vertices):
    if(point[0] >= 0.0 and point[0] <= structure.dimensions["Lx"] and point[1] >= 0.0 and point[1] <= structure.dimensions["Ly"]):
        dist[i] = np.min(distance.cdist(np.array([point]),coordinates.T))
pore_radius = np.max(dist)
print(pore_radius,np.argmax(dist))
pore_center = [vor.vertices[np.argmax(dist)][0], vor.vertices[np.argmax(dist)][1], vor.vertices[np.argmax(dist)][2]]
print('pore center:')
print(pore_center)

9.160155877338036 245
pore center:
[22.555999138136762, 1.3947658574764432e-06, 20.400000000000002]
CPU times: user 195 ms, sys: 535 µs, total: 196 ms
Wall time: 194 ms


In [ ]:
#pore = Pore3D(func, system_size, [64, 64, 64], coordinates * ANGSTROM, sigma, epsilon)
potential = ExternalPotential.FreeEnergyAveraged(coordinates * ANGSTROM, sigma, epsilon, pore_center, system_size, [51, 51])
#pore = Pore1D(func, Geometry.Spherical, 5.9595 * ANGSTROM, potential, 128)
pore = Pore1D(func, Geometry.Cylindrical, 9.160155877 * ANGSTROM, potential, 128)
                                          #pore size?
pressures1 = np.linspace(0.1,1.5,3)
pressures2 = np.linspace(1.7,5.0,3)
# pressures = np.concatenate((pressures1,pressures2), axis=0, out=None)
#pressures3 =np.array([5])
pressures4 = np.linspace(8.0,50.0,3)
pressures = np.concatenate((pressures1,pressures2,pressures4), axis=0, out=None)
isotherm_save = []
for p in pressures:
    s = State(func, temperature = 300.0 * KELVIN, pressure = p * BAR)
    PoreProfile = pore.initialize(s)

    solver = DFTSolver(output=True).picard_iteration(tol=5.0e-5,beta=0.01).picard_iteration(tol=1.0e-7,beta=0.1).anderson_mixing(tol=1.0e-8,mmax=10)
    # Lösen des Dichteprofils
    PoreProfile.solve(solver)

    N_ads = PoreProfile.moles * NAV
    isotherm_save.append(N_ads)

[/usr/ITT/bursik/promotion/entropy_scaling_AS/entropyScaling/feos-dft/src/adsorption/fea_potential.rs:101] weights_sum = 192.26547039969495
[/usr/ITT/bursik/promotion/entropy_scaling_AS/entropyScaling/feos-dft/src/adsorption/fea_potential.rs:138] temperature = 300.0


In [ ]:
isotherm_save_unit = [x[0] * 9.16 *ANGSTROM for x in isotherm_save]
isotherm_save_unit2 = [x[0] *METER for x in isotherm_save]

isotherm_save_unit


In [ ]:
pressures

In [ ]:
isotherm_3D = pd.read_csv('./Adsorptionsisothermen an 13X/anTpPA_CO2_300K.dat', names=['Druck','N_ads'], delim_whitespace=True)
ax = isotherm_3D.plot( x='Druck', y='N_ads', label = '3D')
ax.plot(pressures, isotherm_save_unit, label = '1D')
#ax.plot(pressures, isotherm_save_unit2, label = '1D', color = 'r')
plt.show()
#isotherm_3D